# Data Visualization

In [1]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.neighbors
import matplotlib.pyplot as plt
import ast
import gensim, logging
import xgboost

C:\Users\caioj\Anaconda3\envs\forest-env\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/evaluation.csv')
train_df.describe()

,retweets_count,favorites_count,followers_count,statuses_count,friends_count,verified,timestamp,TweetID
count,353969.000000,353969.000000,3.539690e+05,3.539690e+05,353969.000000,353969.000000,3.539690e+05,3.539690e+05
mean,15.831810,46.655442,2.021548e+04,4.808507e+04,1459.289003,0.030005,1.647004e+12,6.872503e+05
std,241.986723,852.044385,2.598715e+05,1.133854e+05,2502.933271,0.170602,4.846468e+09,4.175793e+05
min,0.000000,0.000000,0.000000e+00,1.000000e+00,0.000000,0.000000,1.301178e+12,3.000000e+00
25%,0.000000,0.000000,1.600000e+02,2.972000e+03,214.000000,0.000000,1.647068e+12,3.194490e+05
50%,1.000000,0.000000,7.260000e+02,1.250100e+04,693.000000,0.000000,1.647292e+12,6.719730e+05
75%,3.000000,1.000000,2.283000e+03,4.352200e+04,1804.000000,0.000000,1.647532e+12,1.049644e+06
max,63674.000000,122591.000000,1.441710e+07,8.183508e+06,237269.000000,1.000000,1.647727e+12,1.434456e+06


In [3]:
train_df.head()

,text,retweets_count,favorites_count,followers_count,statuses_count,friends_count,mentions,urls,verified,hashtags,timestamp,TweetID
0,rt refarcir macron ans nom prépare,3,0,3682,453535,3628,[],[],0,[],1646978048000,832509
1,populaire,0,0,86,1016,284,[],[],0,[],1647694288000,1388011
2,faut dégager cinglé,3,1,1944,28234,1995,[],[],0,[],1647370048000,63896
3,enseignants mettre prescriptions président rép...,0,0,1,1072,0,[],['https://t.co/rytlted08g'],0,[],1647256282000,979251
4,mafieuse oppressive macron,0,0,13957,25311,10841,[],[],0,[],1647258374000,1040049


In [4]:
train_df_input = train_df.drop(['TweetID', 'timestamp', 'mentions', 'retweets_count', 'text'], axis=1)

train_df_input.urls = train_df_input.urls.apply(ast.literal_eval)
train_df_input.urls = train_df_input.urls.apply(len)

train_df_input.hashtags = train_df_input.hashtags.apply(ast.literal_eval)
train_df_input.hashtags = train_df_input.hashtags.apply(len)

train_df_input.head()

,favorites_count,followers_count,statuses_count,friends_count,urls,verified,hashtags
0,0,3682,453535,3628,0,0,0
1,0,86,1016,284,0,0,0
2,1,1944,28234,1995,0,0,0
3,0,1,1072,0,1,0,0
4,0,13957,25311,10841,0,0,0


In [5]:
test_df_input = test_df.drop(['TweetID', 'timestamp', 'mentions', 'text'], axis=1)

test_df_input.urls = test_df_input.urls.apply(ast.literal_eval)
test_df_input.urls = test_df_input.urls.apply(len)

test_df_input.hashtags = test_df_input.hashtags.apply(ast.literal_eval)
test_df_input.hashtags = test_df_input.hashtags.apply(len)

# NLP

In [6]:
nlp_model = gensim.models.Word2Vec.load('models/word2vec.model')

# Trees, Boosting

In [7]:
full_train_X = train_df_input.values
full_train_X = (full_train_X - full_train_X.mean(0)) / full_train_X.std(0)

full_train_y = train_df['retweets_count'].values

full_test_X = (test_df_input.values - train_df_input.values.mean(0)) / train_df_input.values.std(0)

use_text = False

if use_text:
    train_text = train_df['text']
    test_text = test_df['text']

    new_train_X = np.zeros((full_train_X.shape[0], full_train_X.shape[1] + nlp_model.vector_size))
    new_test_X = np.zeros((full_train_X.shape[0], full_train_X.shape[1] + nlp_model.vector_size))

    for i in range(len(full_train_X)):
        text_vec = nlp_model.wv[train_text.iloc[i].split(' ')].mean(0)
        new_train_X[i] = np.concatenate([full_train_X[i], text_vec])

    for i in range(len(full_test_X)):
        encoded_words = [word for word in test_text.iloc[i].split(' ') if word in nlp_model.wv]
        if encoded_words:
            text_vec = nlp_model.wv[encoded_words].mean(0)
            new_test_X[i] = np.concatenate([full_test_X[i], text_vec])
        else:
            new_test_X[i] = np.concatenate([full_test_X[i], np.zeros((nlp_model.vector_size,))])

    full_train_X = new_train_X
    full_test_X = new_test_X

In [ ]:
import sklearn.ensemble
model = sklearn.ensemble.RandomForestRegressor(criterion='absolute_error', verbose=2, max_depth=6)

scores = sklearn.model_selection.cross_validate(model, full_train_X, full_train_y, cv=5,scoring='neg_mean_absolute_error', verbose=2)
- scores['test_score'].mean()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


In [ ]:
# scores_xgboost = sklearn.model_selection.cross_validate(xgboost.XGBRegressor(verbosity=0,
#                                                                              max_depth=10),
#                                                         full_train_X,
#                                                         full_train_y,
#                                                         cv=5,
#                                                         scoring='neg_mean_absolute_error',
#                                                         verbose=2)
#
# - scores_xgboost['test_score'].mean()

# Make submission

In [ ]:
make_submission = False

if make_submission:
    test_ids = test_df[['TweetID']]
    model = xgboost.XGBRegressor(verbosity=2)
    model.fit(full_train_X, full_train_y)

    test_predictions = model.predict(full_test_X)

    submission_df = pd.DataFrame(data={'retweets_count': test_predictions})
    submission_df = pd.concat([test_ids, submission_df], axis=1)
    submission_df.to_csv('data/submission.csv', index=False)